In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_excel("Demand_Forecasting_Monthly_Data.xlsx")
df.drop('Data',axis=1,inplace=True)
df.head(10)

,Orders_Date,Sales_Volume
0,2020-02-01,376832
1,2020-03-01,481754
2,2020-04-01,240192
3,2020-05-01,138704
4,2020-06-01,193440
5,2020-07-01,321818
6,2020-08-01,202245
7,2020-09-01,313422
8,2020-10-01,396599
9,2020-11-01,377673


In [3]:
df.columns

Index(['Orders_Date', 'Sales_Volume'], dtype='object')

In [9]:
import pandas as pd
from statsmodels.tsa.api import Holt

df['Orders_Date'] = pd.to_datetime(df['Orders_Date'])
df = df[df['Orders_Date'] > '2022-04-01']
dates = pd.date_range(start='2023-10-01', end='2024-04-01', freq='MS')

final_result = []

for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Test = df[df['Orders_Date'] == da]
    if Test.empty:
        continue
    Train.set_index('Orders_Date', inplace=True)
    actual_value = Test['Sales_Volume'].values[0]
    model = Holt(
        Train['Sales_Volume'], 
        damped_trend=True, 
        exponential=True, 
        initialization_method="estimated"
    ).fit(smoothing_level=0.2, smoothing_trend=0.3)
    forecast = model.forecast(1)
    predicted_value = forecast.iloc[0]
    data["Orders_Date"] = da
    data['Actual'] = actual_value
    data['Holt-winter'] = predicted_value
    data['Accuracy_SimpleExpo'] = (predicted_value * 100) / actual_value
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

  Orders_Date  Actual    Holt-winter  Accuracy_SimpleExpo
0  2023-10-01  352135  243130.771279            69.044762
1  2023-11-01  349500  268466.416779            76.814425
2  2023-12-01  237928  293604.015443           123.400363
3  2024-01-01  302400  287959.839676            95.224815
4  2024-02-01  287972  297327.709509           103.248826
5  2024-03-01  253875  301408.121383           118.723041
6  2024-04-01  285937  294932.987003           103.146143


C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)
C:\Users\Charan\anaconda3\Lib

In [10]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

Train = df[df['Orders_Date'] <= '2023-09-01']
Train.set_index('Orders_Date', inplace=True)
Train.index = pd.DatetimeIndex(Train.index).to_period('M')
auto_arima(Train['Sales_Volume'],seasonal=True,trace=True,m=4,suppress_warnings=True)

final_result=[]
for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model=SARIMAX(Train['Sales_Volume'],order=(1,0,0),seasonal_order=(0,0,0,4)).fit()
    forecast=model.predict(start=len(Train),end=len(Train)+1,dynamic=True)
    Result = forecast.values[0]
    
    data["SARIMA"] = Result
    data['Accuracy_SimpleExpo'] = ((Result * 100) / Test['Sales_Volume'].values[0])
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

Performing stepwise search to minimize aic
 ARIMA(2,0,2)(1,0,1)[4] intercept   : AIC=inf, Time=1.03 sec
 ARIMA(0,0,0)(0,0,0)[4] intercept   : AIC=432.982, Time=0.03 sec
 ARIMA(1,0,0)(1,0,0)[4] intercept   : AIC=433.831, Time=0.11 sec
 ARIMA(0,0,1)(0,0,1)[4] intercept   : AIC=433.251, Time=0.06 sec
 ARIMA(0,0,0)(0,0,0)[4]             : AIC=478.000, Time=0.00 sec
 ARIMA(0,0,0)(1,0,0)[4] intercept   : AIC=434.863, Time=0.01 sec
 ARIMA(0,0,0)(0,0,1)[4] intercept   : AIC=431.718, Time=0.04 sec
 ARIMA(0,0,0)(1,0,1)[4] intercept   : AIC=433.717, Time=0.04 sec
 ARIMA(0,0,0)(0,0,2)[4] intercept   : AIC=433.716, Time=0.04 sec
 ARIMA(0,0,0)(1,0,2)[4] intercept   : AIC=435.715, Time=0.04 sec
 ARIMA(1,0,0)(0,0,1)[4] intercept   : AIC=432.994, Time=0.02 sec
 ARIMA(1,0,1)(0,0,1)[4] intercept   : AIC=434.358, Time=0.09 sec
 ARIMA(0,0,0)(0,0,1)[4]             : AIC=inf, Time=0.03 sec

Best model:  ARIMA(0,0,0)(0,0,1)[4] intercept
Total fit time: 1.603 seconds
  Orders_Date  Actual         SARIMA  Accur

In [11]:
from prophet import Prophet

dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01','2024-04-01']

final_result=[]
for da in dates:
    data = {}
    Train = df[(df['Orders_Date']>='2022-08-01') & (df['Orders_Date']<da)]
    Train=Train.rename(columns={'Orders_Date' : 'ds', 'Sales_Volume' : 'y'})
    Test=df[(df['Orders_Date']==da)]
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model = Prophet()
    model.fit(Train)
    future=model.make_future_dataframe(periods=2,freq='M').tail(2)
    forecast=model.predict(future)
    forecast[['ds','yhat']]

    Result=sum(forecast['yhat'])/len(forecast)
    data["Prophet"] = Result
    data['Accuracy_SimpleExpo'] = ((Result * 100) / Test['Sales_Volume'].values[0])
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

C:\Users\Charan\anaconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
09:42:02 - cmdstanpy - INFO - Chain [1] start processing
09:42:02 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
09:42:02 - cmdstanpy - INFO - Chain [1] start processing
09:42:02 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
09:42:03 - cmdstanpy - INFO - Chain [1] start processing
09:42:03 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Charan\anac

  Orders_Date  Actual        Prophet  Accuracy_SimpleExpo
0  2023-10-01  352135  252188.941964            71.617119
1  2023-11-01  349500  277156.608546            79.300889
2  2023-12-01  237928  296432.229661           124.589048
3  2024-01-01  302400  284386.003107            94.042990
4  2024-02-01  287972  289376.095718           100.487581
5  2024-03-01  253875  288239.959405           113.536173
6  2024-04-01  285937  283391.728517            99.109849


C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [12]:
from darts import TimeSeries
from darts.models import NBEATSModel
import pandas as pd

dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']

final_result = []
for da in dates:
    data = {}
    Train = df.copy()
    series = TimeSeries.from_dataframe(Train, time_col='Orders_Date', value_cols='Sales_Volume')

    train, val = series.split_before(pd.Timestamp(da))
    Test = df[df['Orders_Date'] == da]
    data['Orders_Date'] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model = NBEATSModel(input_chunk_length=6, output_chunk_length=6, n_epochs=100, activation='ReLU')
    model.fit(train)
    forecast = model.predict(1)
    Result = forecast.pd_series().tolist()

    data["NBEATS"] = Result[0]  # Assuming you want the first value of the forecast
    data['Accuracy_NBEATS'] = ((Result[0] * 100) / Test['Sales_Volume'].values[0])
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 6.1 M  | train
-------------------------------------------------------------
6.1 M     Trainable params
1.3 K     Non-trainable params
6.1 M     Total params
24.217    Total estimated model params size (MB)


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  4.62it/s, train_loss=3.52e+4]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  4.62it/s, train_loss=3.52e+4]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  5.77it/s, train_loss=7.15e+5]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  5.28it/s, train_loss=7.15e+5]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 165.19it/s]
  Orders_Date  Actual         NBEATS  Accuracy_NBEATS
0  2023-10-01  352135  229738.402951        65.241570
1  2023-11-01  349500  181389.135304        51.899610
2  2023-12-01  237928  276722.418562       116.305109
3  2024-01-01  302400  292350.245908        96.676669
4  2024-02-01  287972  248869.685493        86.421487
5  2024-03-01  253875  235300.221913        92.683495
6  2024-04-01  285937  278563.418646        97.421257


In [13]:
from darts import TimeSeries
from darts.models import NHiTSModel
import pandas as pd

dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']

final_result = []
for da in dates:
    data = {}
    Train = df.copy()
    series = TimeSeries.from_dataframe(Train, time_col='Orders_Date', value_cols='Sales_Volume')

    train, val = series.split_before(pd.Timestamp(da))
    Test = df[df['Orders_Date'] == da]
    data['Orders_Date'] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model = NBEATSModel(input_chunk_length=6, output_chunk_length=6, n_epochs=100, activation='ReLU')
    model.fit(train)
    forecast = model.predict(1)
    Result = forecast.pd_series().tolist()

    data["NHiTS"] = Result[0]  # Assuming you want the first value of the forecast
    data['Accuracy_NBEATS'] = ((Result[0] * 100) / Test['Sales_Volume'].values[0])
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | stacks          | ModuleList       | 6.1 M  | train
-------------------------------------------------------------
6.1 M     Trainable params
1.3 K     Non-trainable params
6.1 M     Total params
24.217    Total estimated model params size (MB)


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  7.21it/s, train_loss=1.13e+4]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  7.04it/s, train_loss=1.13e+4]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  2.91it/s, train_loss=1.46e+6]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00,  2.86it/s, train_loss=1.46e+6]

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 58.01it/s] 
  Orders_Date  Actual          NHiTS  Accuracy_NBEATS
0  2023-10-01  352135  226477.932848        64.315655
1  2023-11-01  349500  180509.889531        51.648037
2  2023-12-01  237928  281722.266448       118.406521
3  2024-01-01  302400  296794.144532        98.146212
4  2024-02-01  287972  230316.518999        79.978789
5  2024-03-01  253875  243314.854763        95.840415
6  2024-04-01  285937  297951.905830       104.201942


In [14]:
from darts import TimeSeries
from darts.models import TiDEModel
import pandas as pd

dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']

final_result = []
for da in dates:
    data = {}
    Train = df.copy()
    series = TimeSeries.from_dataframe(Train, time_col='Orders_Date', value_cols='Sales_Volume')

    train, val = series.split_before(pd.Timestamp(da))
    Test = df[df['Orders_Date'] == da]
    data['Orders_Date'] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model = TiDEModel(input_chunk_length=6, output_chunk_length=6, n_epochs=100)
    model.fit(train)
    forecast = model.predict(1)
    Result = forecast.pd_series().tolist()

    data["TiDE"] = Result[0]  # Assuming you want the first value of the forecast
    data['Accuracy_NBEATS'] = ((Result[0] * 100) / Test['Sales_Volume'].values[0])
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name             | Type             | Params | Mode 
--------------------------------------------------------------
0 | criterion        | MSELoss          | 0      | train
1 | train_criterion  | MSELoss          | 0      | train
2 | val_criterion    | MSELoss          | 0      | train
3 | train_metrics    | MetricCollection | 0      | train
4 | val_metrics      | MetricCollection | 0      | train
5 | encoders         | Sequential       | 18.3 K | train
6 | decoders         | Sequential       | 41.3 K | train
7 | temporal_decoder | _ResidualBlock   | 594    | train
8 | lookback_skip    | Linear           | 42     | train
--------------------------------------------------------------
60.2 K    Trainable params
0         Non-trainable params
60.2 K    Total params
0.241     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 56.05it/s, train_loss=3.04e+8] 

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 56.05it/s, train_loss=3.04e+8]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 62.72it/s, train_loss=2.05e+9]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 54.51it/s, train_loss=2.05e+9]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 111.42it/s]
  Orders_Date  Actual           TiDE  Accuracy_NBEATS
0  2023-10-01  352135  212895.027340        60.458355
1  2023-11-01  349500  179938.165214        51.484454
2  2023-12-01  237928  299892.107233       126.043218
3  2024-01-01  302400  318436.894067       105.303206
4  2024-02-01  287972  226825.703318        78.766583
5  2024-03-01  253875  268120.069780       105.611057
6  2024-04-01  285937  303379.076582       106.099972


In [15]:
from darts import TimeSeries
from darts.models import DLinearModel
import pandas as pd

dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']

final_result = []
for da in dates:
    data = {}
    Train = df.copy()
    series = TimeSeries.from_dataframe(Train, time_col='Orders_Date', value_cols='Sales_Volume')

    train, val = series.split_before(pd.Timestamp(da))
    Test = df[df['Orders_Date'] == da]
    data['Orders_Date'] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model = DLinearModel(input_chunk_length=6, output_chunk_length=6, n_epochs=100)
    model.fit(train)
    forecast = model.predict(1)
    Result = forecast.pd_series().tolist()

    data["DLinear"] = Result[0]  # Assuming you want the first value of the forecast
    data['Accuracy_NBEATS'] = ((Result[0] * 100) / Test['Sales_Volume'].values[0])
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | decomposition   | _SeriesDecomp    | 0      | train
6 | linear_seasonal | Linear           | 42     | train
7 | linear_trend    | Linear           | 42     | train
-------------------------------------------------------------
84        Trainable params
0         Non-trainable params
84        Total params
0.000     Total estimated model params size (MB)


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 83.42it/s, train_loss=9.18e+8]   

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 83.42it/s, train_loss=9.18e+8]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 331.83it/s, train_loss=3.46e+9]

`Trainer.fit` stopped: `max_epochs=100` reached.


Epoch 99: 100%|██████████| 1/1 [00:00<00:00, 125.98it/s, train_loss=3.46e+9]


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 74.34it/s] 
  Orders_Date  Actual        DLinear  Accuracy_NBEATS
0  2023-10-01  352135  224680.208409        63.805134
1  2023-11-01  349500  214169.694151        61.278882
2  2023-12-01  237928  269704.063779       113.355328
3  2024-01-01  302400  288206.039787        95.306230
4  2024-02-01  287972  242222.269122        84.113132
5  2024-03-01  253875  273028.281172       107.544375
6  2024-04-01  285937  292846.892618       102.416579


In [16]:
from statsmodels.tsa.holtwinters import SimpleExpSmoothing

df = df[df['Orders_Date'] > '2022-04-01']
final_result = []

for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = SimpleExpSmoothing(Train['Sales_Volume'], initialization_method="estimated").fit(smoothing_level=0.2, optimized=True)
    forecast = model.forecast(1)
    Result = forecast.values[0]
    
    data["SimpleExpo"] = Result
    data['Accuracy_SimpleExpo'] = ((Result * 100) / Test['Sales_Volume'].values[0])
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

  Orders_Date  Actual     SimpleExpo  Accuracy_SimpleExpo
0  2023-10-01  352135  265198.096857            75.311485
1  2023-11-01  349500  282596.474176            80.857360
2  2023-12-01  237928  295982.854230           124.400177
3  2024-01-01  302400  284369.699273            94.037599
4  2024-02-01  287972  287976.817569           100.001673
5  2024-03-01  253875  287975.505897           113.432006
6  2024-04-01  285937  281155.049394            98.327621


In [17]:
from statsmodels.tsa.holtwinters import Holt

df = df[df['Orders_Date'] > '2022-04-01']
final_result = []

for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = Holt(Train['Sales_Volume'], damped_trend=True, initialization_method="estimated").fit(smoothing_level=0.2, smoothing_trend=0.3)
    forecast = model.forecast(1)
    Result = forecast.values[0]
    
    data["Holt"] = Result
    data['Accuracy_Holt'] = ((Result * 100) / Test['Sales_Volume'].values[0])
    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

  Orders_Date  Actual           Holt  Accuracy_Holt
0  2023-10-01  352135  241254.602135      68.511963
1  2023-11-01  349500  266520.165441      76.257558
2  2023-12-01  237928  291103.723421     122.349502
3  2024-01-01  302400  285217.655924      94.318008
4  2024-02-01  287972  294376.342392     102.223946
5  2024-03-01  253875  298380.051451     117.530301
6  2024-04-01  285937  292067.172868     102.143889


C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed

In [18]:
from statsmodels.tsa.api import SimpleExpSmoothing
from prophet import Prophet
dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']

final_result = []
for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = SimpleExpSmoothing(Train['Sales_Volume'], initialization_method="estimated").fit()
    
    forecast = model.forecast(1)
    Result_1 = forecast.values[0]
    
    data["SimpleExpo"] = Result_1

    Train = df[(df['Orders_Date']>='2022-08-01') & (df['Orders_Date']<da)]
    Train=Train.rename(columns={'Orders_Date' : 'ds', 'Sales_Volume' : 'y'})
    Test=df[(df['Orders_Date']==da)]
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model = Prophet()
    model.fit(Train)
    future=model.make_future_dataframe(periods=2,freq='M').tail(2)
    forecast=model.predict(future)
    forecast[['ds','yhat']]

    Result_2=sum(forecast['yhat'])/len(forecast)
    data["Prophet"] = Result_2

    data['Average_All_Algorithms']=(Result_1+Result_2)/2
    data['Accuracy_All_Algorithms']=((((Result_1+Result_2)/2)*100)/Test['Sales_Volume'].values[0])

    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
09:51:51 - cmdstanpy - INFO - Chain [1] start processing
09:51:51 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
09:51:51 - cmdstanpy - INFO - Chain [1] start processing
09:51:51 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\Charan\anaconda3\Lib\si

  Orders_Date  Actual     SimpleExpo        Prophet  Average_All_Algorithms  \
0  2023-10-01  352135  275882.313986  252188.941964           264035.627975   
1  2023-11-01  349500  287840.354228  277156.608546           282498.481387   
2  2023-12-01  237928  294904.355847  296432.229661           295668.292754   
3  2024-01-01  302400  290028.239812  284386.003107           287207.121459   
4  2024-02-01  287972  291087.032311  289376.095718           290231.564015   
5  2024-03-01  253875  290820.443500  288239.959405           289530.201452   
6  2024-04-01  285937  286118.255040  283391.728517           284754.991778   

   Accuracy_All_Algorithms  
0                74.981365  
1                80.829322  
2               124.267969  
3                94.975900  
4               100.784647  
5               114.044392  
6                99.586619  


C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


In [19]:
from statsmodels.tsa.api import SimpleExpSmoothing
from statsmodels.tsa.api import Holt
dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']

final_result = []
for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = SimpleExpSmoothing(Train['Sales_Volume'], initialization_method="estimated").fit()
    
    forecast = model.forecast(1)
    Result_1 = forecast.values[0]
    
    data["SimpleExpo"] = Result_1

    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = Holt(Train['Sales_Volume'],damped_trend=True,exponential=True, initialization_method="estimated").fit(smoothing_level=0.2,smoothing_trend=0.3)
    forecast = model.forecast(1)
    Result_2 = forecast.values[0]
    
    data["Holt"] = Result_2
    data['Average_All_Algorithms']=(Result_1+Result_2)/2
    data['Accuracy_All_Algorithms']=((((Result_1+Result_2)/2)*100)/Test['Sales_Volume'].values[0])

    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

  Orders_Date  Actual     SimpleExpo           Holt  Average_All_Algorithms  \
0  2023-10-01  352135  275882.313986  243130.771279           259506.542633   
1  2023-11-01  349500  287840.354228  268466.416779           278153.385503   
2  2023-12-01  237928  294904.355847  293604.015443           294254.185645   
3  2024-01-01  302400  290028.239812  287959.839676           288994.039744   
4  2024-02-01  287972  291087.032311  297327.709509           294207.370910   
5  2024-03-01  253875  290820.443500  301408.121383           296114.282441   
6  2024-04-01  285937  286118.255040  294932.987003           290525.621021   

   Accuracy_All_Algorithms  
0                73.695186  
1                79.586090  
2               123.673626  
3                95.566812  
4               102.165270  
5               116.637827  
6               101.604766  


C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed

In [20]:
from statsmodels.tsa.api import Holt
from prophet import Prophet
dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01','2024-04-01']

final_result=[]
for da in dates:
    data = {}
    Train = df[(df['Orders_Date']>='2022-08-01') & (df['Orders_Date']<da)]
    Train=Train.rename(columns={'Orders_Date' : 'ds', 'Sales_Volume' : 'y'})
    Test=df[(df['Orders_Date']==da)]
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model = Prophet()
    model.fit(Train)
    future=model.make_future_dataframe(periods=2,freq='M').tail(2)
    forecast=model.predict(future)
    forecast[['ds','yhat']]

    Result_1=sum(forecast['yhat'])/len(forecast)
    data["Prophet"] = Result_1

    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = Holt(Train['Sales_Volume'],damped_trend=True,exponential=True, initialization_method="estimated").fit(smoothing_level=0.2,smoothing_trend=0.3)
    forecast = model.forecast(1)
    Result_2 = forecast.values[0]
    
    data["Holt"] = Result_2
    data['Average_All_Algorithms']=(Result_1+Result_2)/2
    data['Accuracy_All_Algorithms']=((((Result_1+Result_2)/2)*100)/Test['Sales_Volume'].values[0])

    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)
    

09:52:40 - cmdstanpy - INFO - Chain [1] start processing
09:52:40 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
09:52:40 - cmdstanpy - INFO - Chain [1] start processing
09:52:40 - cmdstanpy - INFO - Chain [1] done processing
C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
09:52:40 - cmdstanpy - INFO - Ch

  Orders_Date  Actual        Prophet           Holt  Average_All_Algorithms  \
0  2023-10-01  352135  252188.941964  243130.771279           247659.856622   
1  2023-11-01  349500  277156.608546  268466.416779           272811.512662   
2  2023-12-01  237928  296432.229661  293604.015443           295018.122552   
3  2024-01-01  302400  284386.003107  287959.839676           286172.921391   
4  2024-02-01  287972  289376.095718  297327.709509           293351.902614   
5  2024-03-01  253875  288239.959405  301408.121383           294824.040394   
6  2024-04-01  285937  283391.728517  294932.987003           289162.357760   

   Accuracy_All_Algorithms  
0                70.330940  
1                78.057657  
2               123.994705  
3                94.633903  
4               101.868203  
5               116.129607  
6               101.127996  


C:\Users\Charan\anaconda3\Lib\site-packages\prophet\forecaster.py:1854: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


In [21]:
from statsmodels.tsa.api import SimpleExpSmoothing
from statsmodels.tsa.api import ExponentialSmoothing
dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']

final_result = []
for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = SimpleExpSmoothing(Train['Sales_Volume'], initialization_method="estimated").fit()
    
    forecast = model.forecast(1)
    Result_1 = forecast.values[0]
    
    data["SimpleExpo"] = Result_1
    
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = ExponentialSmoothing(Train['Sales_Volume'],seasonal_periods=4,trend="add", seasonal="mul",damped_trend=True,use_boxcox=True,initialization_method="estimated").fit()
    forecast = model.forecast(1)
    Result_2 = forecast.values[0]
    
    data["Holt-winter"] = Result_2
    data['Average_All_Algorithms']=(Result_1+Result_2)/2
    data['Accuracy_All_Algorithms']=((((Result_1+Result_2)/2)*100)/Test['Sales_Volume'].values[0])

    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)
    

C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed

  Orders_Date  Actual     SimpleExpo    Holt-winter  Average_All_Algorithms  \
0  2023-10-01  352135  275882.313986  258033.492972           266957.903479   
1  2023-11-01  349500  287840.354228  261814.264105           274827.309166   
2  2023-12-01  237928  294904.355847  236334.246631           265619.301239   
3  2024-01-01  302400  290028.239812  244304.948748           267166.594280   
4  2024-02-01  287972  291087.032311  280195.321254           285641.176783   
5  2024-03-01  253875  290820.443500  265252.656678           278036.550089   
6  2024-04-01  285937  286118.255040  218589.262131           252353.758585   

   Accuracy_All_Algorithms  
0                75.811238  
1                78.634423  
2               111.638521  
3                88.348741  
4                99.190608  
5               109.517105  
6                88.255021  


In [22]:
from statsmodels.tsa.api import SimpleExpSmoothing
from statsmodels.tsa.statespace.sarimax import SARIMAX
from pmdarima import auto_arima

dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01', '2024-04-01']

final_result = []
for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = SimpleExpSmoothing(Train['Sales_Volume'], initialization_method="estimated").fit()
    
    forecast = model.forecast(1)
    Result_1 = forecast.values[0]
    
    data["SimpleExpo"] = Result_1
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]

    model=SARIMAX(Train['Sales_Volume'],order=(1,0,0),seasonal_order=(0,0,0,4)).fit()
    forecast=model.predict(start=len(Train),end=len(Train)+1,dynamic=True)
    Result_2 = forecast.values[0]
    
    data["SARIMA"] = Result_2
    data['Average_All_Algorithms']=(Result_1+Result_2)/2
    data['Accuracy_All_Algorithms']=((((Result_1+Result_2)/2)*100)/Test['Sales_Volume'].values[0])

    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)

  Orders_Date  Actual     SimpleExpo         SARIMA  Average_All_Algorithms  \
0  2023-10-01  352135  275882.313986  288167.670899           282024.992442   
1  2023-11-01  349500  287840.354228  326367.614075           307103.984151   
2  2023-12-01  237928  294904.355847  325566.378838           310235.367343   
3  2024-01-01  302400  290028.239812  219222.818617           254625.529214   
4  2024-02-01  287972  291087.032311  280973.441529           286030.236920   
5  2024-03-01  253875  290820.443500  268169.561973           279495.002736   
6  2024-04-01  285937  286118.255040  236391.527097           261254.891068   

   Accuracy_All_Algorithms  
0                80.090020  
1                87.869523  
2               130.390441  
3                84.201564  
4                99.325711  
5               110.091582  
6                91.367991  


C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(
C:\Users\Charan\anaconda3\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed

In [ ]:
from statsmodels.tsa.api import Holt
from statsmodels.tsa.api import ExponentialSmoothing
dates = ['2023-10-01', '2023-11-01', '2023-12-01', '2024-01-01', '2024-02-01', '2024-03-01','2024-04-01']

final_result=[]
for da in dates:
    data = {}
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = Holt(Train['Sales_Volume'],damped_trend=True,exponential=True, initialization_method="estimated").fit(smoothing_level=0.2,smoothing_trend=0.3)
    forecast = model.forecast(1)
    Result_1 = forecast.values[0]
    
    data["Holt"] = Result_1
    Train = df[df['Orders_Date'] < da]
    Train.set_index('Orders_Date', inplace=True)
    Train.index = pd.DatetimeIndex(Train.index).to_period('M')
    Test = df[df['Orders_Date'] == da]
    
    data["Orders_Date"] = da
    data['Actual'] = Test['Sales_Volume'].values[0]
    
    model = ExponentialSmoothing(Train['Sales_Volume'],seasonal_periods=4,trend="add", seasonal="mul",damped_trend=True,use_boxcox=True,initialization_method="estimated").fit()
    forecast = model.forecast(1)
    Result_2 = forecast.values[0]
    
    data["Holt-winter"] = Result_2
    data['Average_All_Algorithms']=(Result_1+Result_2)/2
    data['Accuracy_All_Algorithms']=((((Result_1+Result_2)/2)*100)/Test['Sales_Volume'].values[0])

    final_result.append(data)

final_TotalOrders = pd.DataFrame(final_result)
print(final_TotalOrders)